In [2]:
import cv2
import numpy as np
from action_package import ROBOT
import time
import threading
import ctypes
import inspect
import ipywidgets.widgets as widgets
from IPython.display import display

# Robot object
robot = ROBOT()

# Camera configuration
cam = cv2.VideoCapture(0, cv2.CAP_V4L2)
cam.set(3,320)
cam.set(4,240)

# Thread stop code
def _async_raise(tid, exctype):
    tid = ctypes.c_long(tid)
    if not inspect.isclass(exctype):
        exctype = type(exctype)
    res = ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, ctypes.py_object(exctype))
    if res == 0:
        raise ValueError("invalid thread id")
    elif res != 1:
        ctypes.pythonapi.PyThreadState_SetAsyncExc(tid, None)
        raise SystemError("PyThreadState_SetAsyncExc failed")

def stop_thread(thread):
    _async_raise(thread.ident, SystemExit)

# Convert BGR image to JPEG
def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

# Initializes the display area
FGmaskComp_img = widgets.Image(format='jpeg', width=320, height=240)
frame_img = widgets.Image(format='jpeg', width=320, height=240)

# overall arrangement
dispaly_img = widgets.HBox([FGmaskComp_img, frame_img])
display(dispaly_img)

# Video display and processing
def Video_display():
    last_gesture = None  # It is used to record the last detected gesture to avoid repeated operations
    last_move_time = time.time()  # Record the last time a robot action was performed
    gesture_duration = 0.5  # Set the time interval for gesture detection (seconds)
    detection_interval = 3  # Gestures are detected every 3 seconds

    while True:
        ret, frame = cam.read()
        if not ret:
            break

        frame = cv2.flip(frame, 0)
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Color range (skin tone range, palm color) :
        lower_skin = np.array([0, 20, 70], dtype=np.uint8)  # Lower bound for skin color
        upper_skin = np.array([20, 255, 255], dtype=np.uint8)  # Upper bound for skin color

        # color filtration
        mask = cv2.inRange(hsv, lower_skin, upper_skin)
        FGmaskComp_img.value = bgr8_to_jpeg(mask)

        # contour detection
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

        gesture = None  # By default, no gesture is detected
        # If we detect the palm area
        for cnt in contours:
            area = cv2.contourArea(cnt)
            (x, y, w, h) = cv2.boundingRect(cnt)
            if area >= 500:  # Set a threshold to avoid detecting areas that are too small
                cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 3)  # Marked palm area
                roi = mask[y:y + h, x:x + w]

                # Convex Hull and Convex Defects are used to identify fingers
                hull = cv2.convexHull(cnt)
                
                # Ensure that the contour has enough points to calculate convexity defects
                if len(hull) >= 3:  # Minimum 3 points for a valid convex hull
                    try:
                        defects = cv2.convexityDefects(cnt, cv2.convexHull(cnt, returnPoints=False))

                        # Finger count
                        finger_count = 0
                        if defects is not None:
                            for i in range(defects.shape[0]):
                                s, e, f, d = defects[i, 0]
                                start = tuple(cnt[s][0])
                                end = tuple(cnt[e][0])
                                far = tuple(cnt[f][0])

                                # Calculate the Angle and determine if it's the tip of your finger
                                a = np.linalg.norm(np.array(start) - np.array(far))
                                b = np.linalg.norm(np.array(end) - np.array(far))
                                c = np.linalg.norm(np.array(start) - np.array(end))

                                angle = np.arccos((a ** 2 + b ** 2 - c ** 2) / (2 * a * b)) * 57

                                if angle <= 90:  # Less than 90 degrees is a sharp Angle, indicating fingers
                                    finger_count += 1
                                    cv2.line(frame, start, far, [0, 255, 0], 2)
                                    cv2.line(frame, end, far, [0, 255, 0], 2)
                    except cv2.error as e:
                        print(f"Error with convexity defects: {e}")
                        continue  # Skip this contour if there's an error

                # Judging gesture
                if finger_count == 1:
                    gesture = "Left"
                elif finger_count == 2:
                    gesture = "Right"
                elif finger_count == 3:
                    gesture = "Forward"
                elif finger_count == 4:
                    gesture = "Backward"
                elif finger_count == 5:
                    gesture = "Stop"
                break  # Stop further processing when you find the first palm (assuming a palm)

        # If 3 seconds have passed since the last detection, detect the gesture
        if time.time() - last_move_time > detection_interval:
            if gesture != last_gesture:
                if gesture == "Left":
                    robot.moveLeft(30, 1)
                    last_move_time = time.time()  # Update robot movement time
                elif gesture == "Right":
                    robot.moveRight(30, 1)
                    last_move_time = time.time()  
                elif gesture == "Forward":
                    robot.t_up(30, 1)
                    last_move_time = time.time()  
                elif gesture == "Backward":
                    robot.t_down(30, 1)
                    last_move_time = time.time()  
                elif gesture == "Stop":
                    robot.t_stop(0)  # Stop robot motion
                    last_move_time = time.time()  # Update stop time

            last_gesture = gesture  # Update the last detected gesture

        # Check if the robot does not receive a new gesture within the specified time, and stop the robot if the time exceeds
        if time.time() - last_move_time > gesture_duration:
            robot.t_stop(0)

        frame_img.value = bgr8_to_jpeg(frame)  # Show current frame

    cam.release()

t = threading.Thread(target=Video_display)
t.setDaemon(True)
t.start()


[ WARN:1] global ../modules/videoio/src/cap_v4l.cpp (893) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


In [26]:
stop_thread(t)

ValueError: invalid thread id

In [ ]:
# Convex Hull and Convex Defects are used to identify fingers
hull = cv2.convexHull(cnt)
# Ensure that the contour has enough points to calculate convexity defects
if len(hull) >= 3:  # Minimum 3 points for a valid convex hull
    try:
        defects = cv2.convexityDefects(cnt, cv2.convexHull(cnt, returnPoints=False))
        # Finger count
        finger_count = 0
        if defects is not None:
            for i in range(defects.shape[0]):
                s, e, f, d = defects[i, 0]
                start = tuple(cnt[s][0])
                end = tuple(cnt[e][0])
                far = tuple(cnt[f][0])
                # Calculate the Angle and determine if it's the tip of your finger
                a = np.linalg.norm(np.array(start) - np.array(far))
                b = np.linalg.norm(np.array(end) - np.array(far))
                c = np.linalg.norm(np.array(start) - np.array(end))
                angle = np.arccos((a ** 2 + b ** 2 - c ** 2) / (2 * a * b)) * 57
                if angle <= 90:  # Less than 90 degrees is a sharp Angle, indicating fingers
                    finger_count += 1